# Чат бот с искусственным интелектом

### Подключение библиотек

In [14]:
import random
import nltk
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer 

import logging
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

### Открытие BOT_CONFIG.json

In [15]:
with open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

### Заполяем данные для модели

In [ ]:
X = []
Y = []

for intent in BOT_CONFIG['intents'].keys():
  try:
    for example in BOT_CONFIG['intents'][intent]['examples']:
      X.append(example)
      Y.append(intent)
  except:
    print(BOT_CONFIG['intents'][intent])

### Тренировка модели, чтобы избежать переобучения

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

### Применяем векторизацию для всех слов из словаря

In [18]:
vectorizer = CountVectorizer(ngram_range=(1, 5), analyzer='word')
X_train_vectorized = vectorizer.fit_transform(X_train)

### Обучаем модель

In [19]:
clf = LogisticRegression( C=2.0, fit_intercept=False).fit(X_train_vectorized, y_train)
clf.score(X_train_vectorized, y_train)

0.8100320170757738

In [20]:
clf.predict(vectorizer.transform(['ку']))

array(['hello'], dtype='<U25')

In [21]:
def get_intent_by_model(text):
  return clf.predict(vectorizer.transform([text]))[0]

### Установка бота

In [ ]:
!pip install python-telegram-bot --upgrade

## Вызов бота

In [22]:
def bot(input_text):
  intent = get_intent_by_model(input_text)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
input_text = ''
while input_text != 'Завершите работу':
  input_text = input()
  print(bot(input_text))

### Активация бота

In [27]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1936887849:AAHp1oiwXR3GIehamNOoEvAGI36q4UE0AnQ")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
 main()